<a href="https://colab.research.google.com/github/smoreira/TensorFlow2/blob/master/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

In [2]:
%tensorflow_version 2.x
import tensorflow as tf
print("You are using TensorFlow version", tf.__version__)
if len(tf.config.list_physical_devices('GPU')) > 0:
  print("You have a GPU enabled.")
else:
  print("Enable a GPU before running this notebook.")

You are using TensorFlow version 2.2.0-rc3
Enable a GPU before running this notebook.


### Upload Dataset Titanic Disaster

In [4]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving titanic_test.csv to titanic_test.csv
Saving titanic_train.csv to titanic_train.csv
User uploaded file "titanic_test.csv" with length 28629 bytes
User uploaded file "titanic_train.csv" with length 61194 bytes


In [0]:
train_data = pd.read_csv('titanic_train.csv')
test_data = pd.read_csv('titanic_test.csv')

In [0]:
train_data = train_data.join(pd.get_dummies(train_data['Sex']))
test_data = test_data.join(pd.get_dummies(test_data['Sex']))

In [0]:
train_data.loc[train_data['Embarked'].isnull(), 'Embarked'] = 'S'

In [0]:
embarked_encoder = LabelEncoder()
train_data['Embarked'] = embarked_encoder.fit_transform(train_data['Embarked'])
test_data['Embarked'] = embarked_encoder.transform(test_data['Embarked'])

In [0]:
train_data_male_age = train_data.loc[(train_data['Age'].notnull() & train_data['male'] == 1), 'Age']
test_data_male_age = test_data.loc[(test_data['Age'].notnull() & test_data['male'] == 1), 'Age']
male_avg_age = pd.concat([train_data_male_age, test_data_male_age]).mean()

train_data_female_age = train_data.loc[(train_data['Age'].notnull() & train_data['female'] == 1), 'Age']
test_data_female_age = test_data.loc[(test_data['Age'].notnull() & test_data['female'] == 1), 'Age']
female_avg_age = pd.concat([train_data_female_age, test_data_female_age]).mean()

train_data.loc[(train_data['Age'].isnull()) & (train_data['male'] == 1), 'Age'] = male_avg_age
test_data.loc[(test_data['Age'].isnull()) & (test_data['male'] == 1), 'Age'] = male_avg_age

train_data.loc[(train_data['Age'].isnull()) & (train_data['female'] == 1), 'Age'] = female_avg_age
test_data.loc[(test_data['Age'].isnull()) & (test_data['female'] == 1), 'Age'] = female_avg_age

In [0]:
test_data.loc[(test_data['Fare'].isnull())] = 14.5000


In [0]:
train_data.loc[(train_data['SibSp'] == 0) & (train_data['Parch'] == 0), 'Is_alone'] = 1
test_data.loc[(test_data['SibSp'] == 0) & (test_data['Parch'] == 0), 'Is_alone'] = 1

train_data.loc[(train_data['SibSp'] > 0) | (train_data['Parch'] > 0), 'Is_alone'] = 0
test_data.loc[(test_data['SibSp'] > 0) | (test_data['Parch'] > 0), 'Is_alone'] = 0

In [12]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,female,male,Is_alone
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,2,0,1,0.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,0,1,0,0.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,2,1,0,1.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,2,1,0,0.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,2,0,1,1.0


In [0]:
test_data_passenger_id = test_data['PassengerId']

columns_to_drop = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'Sex', 'SibSp', 'Parch']
train_data.drop(columns=columns_to_drop, inplace=True)
train_data_columns = train_data.columns

test_data.drop(columns=columns_to_drop, inplace=True)
test_data_columns = test_data.columns

In [14]:
train_data.head()

,Survived,Pclass,Age,Fare,Embarked,female,male,Is_alone
0,0,3,22.0,7.2500,2,0,1,0.0
1,1,1,38.0,71.2833,0,1,0,0.0
2,1,3,26.0,7.9250,2,1,0,1.0
3,1,1,35.0,53.1000,2,1,0,0.0
4,0,3,35.0,8.0500,2,0,1,1.0


In [0]:
age_scaler = MinMaxScaler()
train_data['Age'] = age_scaler.fit_transform(train_data['Age'].values.reshape(-1, 1))
test_data['Age'] = age_scaler.transform(test_data['Age'].values.reshape(-1, 1))

fare_scaler = MinMaxScaler()
train_data['Fare'] = fare_scaler.fit_transform(train_data['Fare'].values.reshape(-1, 1))
test_data['Fare'] = fare_scaler.transform(test_data['Fare'].values.reshape(-1, 1))

train_data = pd.DataFrame(train_data)
train_data.columns = train_data_columns

test_data = pd.DataFrame(test_data)
test_data.columns = test_data_columns

In [0]:
train_data_label = train_data['Survived']
train_data.drop(columns=['Survived'], inplace=True)

In [17]:
train_data.head()

,Pclass,Age,Fare,Embarked,female,male,Is_alone
0,3,0.271174,0.014151,2,0,1,0.0
1,1,0.472229,0.139136,0,1,0,0.0
2,3,0.321438,0.015469,2,1,0,1.0
3,1,0.434531,0.103644,2,1,0,0.0
4,3,0.434531,0.015713,2,0,1,1.0


In [0]:
train_data = train_data.values
train_data_label = train_data_label.values

test_data = test_data.values

In [20]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(45, input_dim=7, activation='relu'))
model.add(tf.keras.layers.Dense(30, activation='relu'))
model.add(tf.keras.layers.Dense(20, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='relu'))

model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_data, train_data_label, batch_size=100, epochs=150, verbose=1)

loss, accuracy = model.evaluate(train_data, train_data_label)
print(accuracy)

Epoch 1/150
9/9 [==============================] - 0s 2ms/step - loss: 0.6593 - accuracy: 0.6162
Epoch 2/150
9/9 [==============================] - 0s 2ms/step - loss: 0.6252 - accuracy: 0.6162
Epoch 3/150
9/9 [==============================] - 0s 2ms/step - loss: 0.5949 - accuracy: 0.6173
Epoch 4/150
9/9 [==============================] - 0s 2ms/step - loss: 0.5710 - accuracy: 0.6633
Epoch 5/150
9/9 [==============================] - 0s 2ms/step - loss: 0.5487 - accuracy: 0.7273
Epoch 6/150
9/9 [==============================] - 0s 2ms/step - loss: 0.5330 - accuracy: 0.7924
Epoch 7/150
9/9 [==============================] - 0s 2ms/step - loss: 0.5196 - accuracy: 0.7946
Epoch 8/150
9/9 [==============================] - 0s 2ms/step - loss: 0.5087 - accuracy: 0.8081
Epoch 9/150
9/9 [==============================] - 0s 2ms/step - loss: 0.5003 - accuracy: 0.7991
Epoch 10/150
9/9 [==============================] - 0s 2ms/step - loss: 0.4887 - accuracy: 0.7969
Epoch 11/150
9/9 [===========